In [ ]:

def safe_load_results(experiment_path):
    """
    Load results saved with safe_save_results
    """
    print(f"Loading results from: {experiment_path}")
    
    # Load metadata
    try:
        with open(os.path.join(experiment_path, 'metadata.json'), 'r') as f:
            metadata = json.load(f)
        print(f"Experiment: {metadata['experiment_name']}")
        print(f"Models: {metadata['models']}")
    except:
        print("Warning: Could not load metadata")
        # Try to find model directories
        metadata = {'models': [d for d in os.listdir(experiment_path) 
                              if os.path.isdir(os.path.join(experiment_path, d))]}
    
    results = {}
    
    for model_name in metadata['models']:
        model_path = os.path.join(experiment_path, model_name)
        print(f"  Loading {model_name}...")
        
        model_results = {}
        
        # Load models
        try:
            with open(os.path.join(model_path, 'model.pkl'), 'rb') as f:
                model_results['model'] = pickle.load(f)
            with open(os.path.join(model_path, 'calibrated_model.pkl'), 'rb') as f:
                model_results['calibrated_model'] = pickle.load(f)
            print(f"    ✅ Models loaded")
        except Exception as e:
            print(f"    ❌ Could not load models: {e}")
        
        # Load arrays
        try:
            model_results['predictions'] = np.load(os.path.join(model_path, 'predictions.npy'))
            model_results['probabilities_raw'] = np.load(os.path.join(model_path, 'probabilities_raw.npy'))
            model_results['probabilities_calibrated'] = np.load(os.path.join(model_path, 'probabilities_calibrated.npy'))
            print(f"    ✅ Arrays loaded")
        except Exception as e:
            print(f"    ❌ Could not load arrays: {e}")
        
        # Load metrics
        try:
            with open(os.path.join(model_path, 'metrics.json'), 'r') as f:
                metrics = json.load(f)
            model_results.update(metrics)
            print(f"    ✅ Metrics loaded")
        except Exception as e:
            print(f"    ❌ Could not load metrics: {e}")
        
        # Load calibration metrics
        try:
            cal_path = os.path.join(model_path, 'calibration')
            
            with open(os.path.join(cal_path, 'metrics.json'), 'r') as f:
                cal_metrics = json.load(f)
            
            # Load calibration curves if they exist
            try:
                cal_metrics['calibration_curve_raw'] = {
                    'fraction_positives': np.load(os.path.join(cal_path, 'curve_raw_frac.npy')),
                    'mean_predicted': np.load(os.path.join(cal_path, 'curve_raw_mean.npy'))
                }
            except:
                cal_metrics['calibration_curve_raw'] = None
            
            try:
                cal_metrics['calibration_curve_calibrated'] = {
                    'fraction_positives': np.load(os.path.join(cal_path, 'curve_cal_frac.npy')),
                    'mean_predicted': np.load(os.path.join(cal_path, 'curve_cal_mean.npy'))
                }
            except:
                cal_metrics['calibration_curve_calibrated'] = None
            
            model_results['calibration_metrics'] = cal_metrics
            print(f"    ✅ Calibration metrics loaded")
        except Exception as e:
            print(f"    ❌ Could not load calibration metrics: {e}")
        
        # Load risk scores
        try:
            risk_path = os.path.join(model_path, 'risk_scores')
            
            risk_scores = {}
            risk_scores['max_probability'] = np.load(os.path.join(risk_path, 'max_probability.npy'))
            risk_scores['predicted_class'] = np.load(os.path.join(risk_path, 'predicted_class.npy'))
            risk_scores['class_probabilities'] = np.load(os.path.join(risk_path, 'class_probabilities.npy'))
            
            # Load risk categories
            risk_cats_df = pd.read_csv(os.path.join(risk_path, 'risk_categories.csv'))
            risk_scores['risk_category'] = risk_cats_df.iloc[:, 0].values
            
            # Load risk performance if available
            try:
                with open(os.path.join(risk_path, 'risk_performance.json'), 'r') as f:
                    risk_scores['risk_performance'] = json.load(f)
            except:
                pass
            
            model_results['risk_scores'] = risk_scores
            print(f"    ✅ Risk scores loaded")
        except Exception as e:
            print(f"    ❌ Could not load risk scores: {e}")
        
        # Load feature importance
        try:
            feat_path = os.path.join(model_path, 'feature_importance')
            feat_importance = {}
            
            if os.path.exists(feat_path):
                # Get all files in the feature importance directory
                feat_files = os.listdir(feat_path)
                
                for file_name in feat_files:
                    file_path = os.path.join(feat_path, file_name)
                    base_name = os.path.splitext(file_name)[0]
                    
                    try:
                        if file_name.endswith('.csv'):
                            # Try to load as DataFrame first, then as Series if it fails
                            try:
                                feat_importance[base_name] = pd.read_csv(file_path)
                            except:
                                feat_importance[base_name] = pd.read_csv(file_path, index_col=0).iloc[:, 0]
                        
                        elif file_name.endswith('.npy'):
                            feat_importance[base_name] = np.load(file_path)
                        
                        elif file_name.endswith('.json'):
                            with open(file_path, 'r') as f:
                                feat_importance[base_name] = json.load(f)
                        
                        elif file_name.endswith('.pkl'):
                            with open(file_path, 'rb') as f:
                                feat_importance[base_name] = pickle.load(f)
                    
                    except Exception as sub_e:
                        print(f"      Warning: Could not load {file_name}: {sub_e}")
            
            model_results['feature_importance'] = feat_importance
            print(f"    ✅ Feature importance loaded ({len(feat_importance)} components)")
        except Exception as e:
            print(f"    ❌ Could not load feature importance: {e}")
            model_results['feature_importance'] = {}
        
        results[model_name] = model_results
    
    print(f"\n🎉 Results loaded successfully!")
    return results


    def predict_single_patient_risk(patient_data, trained_results, model_name='best_model'):
    """
    Predict risk score for a single patient using calibrated probabilities.
    
    Parameters:
    -----------
    patient_data : pandas.DataFrame or numpy.array
        Single patient's feature data (1 row)
    trained_results : dict
        Results from train_and_evaluate_models_with_calibration()
    model_name : str
        Name of the model to use for prediction
    
    Returns:
    --------
    dict : Comprehensive risk assessment for the patient
    """
    
    # Get the calibrated model
    if model_name == 'best_model':
        # Automatically select best performing model based on calibrated AUC
        best_model = max(trained_results.keys(), 
                        key=lambda x: trained_results[x].get('test_auc_calibrated', 0))
        model_name = best_model
    
    calibrated_model = trained_results[model_name]['calibrated_model']
    
    # Get predictions
    raw_probabilities = trained_results[model_name]['model'].predict_proba(patient_data)
    calibrated_probabilities = calibrated_model.predict_proba(patient_data)
    predicted_class = calibrated_model.predict(patient_data)
    
    # Calculate risk scores
    max_prob = np.max(calibrated_probabilities, axis=1)[0]
    predicted_class_idx = predicted_class[0]
    
    # Risk categorization
    if max_prob >= 0.8:
        risk_category = "High Risk"
        risk_description = "Strong evidence for PID - immediate clinical attention recommended"
    elif max_prob >= 0.6:
        risk_category = "Medium Risk"  
        risk_description = "Moderate evidence for PID - further evaluation recommended"
    elif max_prob >= 0.4:
        risk_category = "Low Risk"
        risk_description = "Limited evidence for PID - consider monitoring"
    else:
        risk_category = "Very Low Risk"
        risk_description = "Minimal evidence for PID - routine follow-up"
    
    # Create comprehensive risk report
    risk_report = {
        'patient_id': f"Patient_{hash(str(patient_data.iloc[0].values)) % 10000}",
        'model_used': model_name,
        
        # Main Risk Assessment
        'risk_score': round(max_prob * 100, 1),  # Convert to percentage
        'risk_category': risk_category,
        'risk_description': risk_description,
        'predicted_class': predicted_class_idx,
        
        # Detailed Probabilities
        'calibrated_probabilities': {
            f'Class_{i}': round(prob * 100, 1) 
            for i, prob in enumerate(calibrated_probabilities[0])
        },

        'raw_probabilities': {
            f'Class_{i}': round(prob * 100, 1) 
            for i, prob in enumerate(raw_probabilities[0])
        },
        
        # Clinical Interpretation
        'confidence_level': get_confidence_interpretation(max_prob),
        'recommendation': get_clinical_recommendation(max_prob, predicted_class_idx),

         ## Top 3 imp features based on impotance df 
        'top Features': trained_results[model_name]['feature_importance']['importance_df'].head(3).to_dict(orient='records'),

        
        
        # Model Performance Context
        'model_performance': {
            # 'model_accuracy': round(trained_results[model_name]['test_accuracy'] * 100, 1),
            # 'model_f1_score': round(trained_results[model_name]['test_f1'] * 100, 1),
            # 'calibration_quality': get_calibration_quality(trained_results[model_name]),
            # 'accuracy_out_of_10': get_accuracy_out_of_10(trained_results[model_name]['test_accuracy']),
            # 'precision_out_of_10': get_accuracy_out_of_10(trained_results[model_name]['test_precision']),
            # 'recall_out_of_10': get_accuracy_out_of_10(trained_results[model_name]['test_recall']),
            'balanced_accuracy': get_accuracy_out_of_10(trained_results[model_name]['test_balanced_accuracy']),
        }
    }
    
    return risk_report


def get_accuracy_out_of_10(metric_value):
    """Convert decimal accuracy/precision/recall to 'out of 10' format."""
    out_of_10 = round(metric_value * 10)
    return f"{out_of_10} out of 10 Patients are predicted correctly"


def get_detailed_performance_interpretation(metric_value, metric_name):
    """Provide detailed interpretation of model performance metrics."""
    out_of_10 = round(metric_value * 10)
    percentage = round(metric_value * 100, 1)
    
    interpretations = {
        'accuracy': f"Correctly predicts {out_of_10} out of 10 cases overall",
        'precision': f"When predicting PID, {out_of_10} out of 10 predictions are correct",
        'recall': f"Identifies {out_of_10} out of 10 actual PID cases",
        'f1': f"Balanced performance score: {percentage}%"
    }
    
    return {
        'out_of_10': f"{out_of_10} out of 10",
        'percentage': f"{percentage}%",
        'interpretation': interpretations.get(metric_name, f"{out_of_10} out of 10")
    }
def get_confidence_interpretation(probability):
    
    """Convert probability to confidence interpretation."""
    if probability >= 0.9:
        return "Very High Confidence"
    elif probability >= 0.8:
        return "High Confidence"
    elif probability >= 0.7:
        return "Moderate Confidence"
    elif probability >= 0.6:
        return "Fair Confidence"
    else:
        return "Low Confidence"


def get_clinical_recommendation(probability, predicted_class):
    """Provide clinical recommendations based on risk score."""
    if probability >= 0.8:
        return "Immediate clinical evaluation recommended. Consider antibiotic treatment."
    elif probability >= 0.6:
        return "Further diagnostic testing recommended (e.g., pelvic exam, lab tests)."
    elif probability >= 0.4:
        return "Monitor symptoms closely. Consider follow-up if symptoms persist."
    else:
        return "Routine follow-up. Educate patient on when to seek care."


def get_calibration_quality(model_results):
    """Assess calibration quality of the model."""
    brier_improvement = model_results['calibration_metrics'].get('brier_improvement', 0)
    if brier_improvement > 0.05:
        return "Excellent calibration improvement"
    elif brier_improvement > 0.02:
        return "Good calibration improvement"
    elif brier_improvement > 0.01:
        return "Fair calibration improvement"
    else:
        return "Minimal calibration improvement"


def print_patient_risk_report(risk_report):
    """Print a formatted risk report for a patient."""
    print("="*80)
    print(f"PID RISK ASSESSMENT REPORT")
    print("="*80)
    print(f"Patient ID: {risk_report['patient_id']}")
    print(f"Model Used: {risk_report['model_used']}")
    print()
    
    print("RISK ASSESSMENT:")
    print(f"  Risk Score: {risk_report['risk_score']}%")
    print(f"  Risk Category: {risk_report['risk_category']}")
    print(f"  Confidence: {risk_report['confidence_level']}")
    print()
    
    print("CLINICAL INTERPRETATION:")
    print(f"  {risk_report['risk_description']}")
    print()
    
    print("RECOMMENDATION:")
    print(f"  {risk_report['recommendation']}")
    print()
    
    print("DETAILED PROBABILITIES:")
    print("  Calibrated Probabilities:")
    for class_name, prob in risk_report['calibrated_probabilities'].items():
        print(f"    {class_name}: {prob}%")
    print()

# Top 3 Features
    print("\n🔍 Top 3 Important Features:")
    print(f"{'Rank':<5} {'Feature':<20} {'Importance':<10}")
    print("-" * 40)
    for idx, feature in enumerate(risk_report['top Features'], start=1):
        print(f"{idx:<5} {feature['feature']:<20} {feature['importance']:<10.4f}")
    
    print("MODEL PERFORMANCE CONTEXT:")
    perf = risk_report['model_performance']
    print(f"balanced_accuracy: {perf['balanced_accuracy']}")
    # print(f"  Model Accuracy: {perf['model_accuracy']}%")
    # print(f"  Model F1-Score: {perf['model_f1_score']}%")
    # print(f"  Calibration Quality: {perf['calibration_quality']}")
    print("="*80)


def batch_predict_patients(patients_data, trained_results, model_name='best_model'):
    """
    Predict risk scores for multiple patients.
    
    Parameters:
    -----------
    patients_data : pandas.DataFrame
        Multiple patients' feature data
    trained_results : dict
        Results from train_and_evaluate_models_with_calibration()
    model_name : str
        Name of the model to use for prediction
    
    Returns:
    --------
    list : List of risk reports for each patient
    """
    risk_reports = []
    
    for idx in range(len(patients_data)):
        patient_data = patients_data.iloc[idx:idx+1]  # Keep as DataFrame
        risk_report = predict_single_patient_risk(patient_data, trained_results, model_name)
        risk_report['patient_index'] = idx
        risk_reports.append(risk_report)
    
    return risk_reports


def create_risk_summary_dashboard(risk_reports):
    """Create a summary dashboard of multiple patient risk assessments."""
    import pandas as pd
    
    # Extract key metrics
    summary_data = []
    for report in risk_reports:
        summary_data.append({
            'Patient_ID': report['patient_id'],
            'Risk_Score': report['risk_score'],
            'Risk_Category': report['risk_category'],
            'Confidence': report['confidence_level'],
            'Predicted_Class': report['predicted_class']
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Risk distribution
    risk_distribution = summary_df['Risk_Category'].value_counts()
    
    print("PATIENT RISK SUMMARY DASHBOARD")
    print("="*60)
    print(f"Total Patients Assessed: {len(risk_reports)}")
    print()
    print("Risk Distribution:")
    for category, count in risk_distribution.items():
        percentage = (count / len(risk_reports)) * 100
        print(f"  {category}: {count} patients ({percentage:.1f}%)")
    print()
    
    print("High Risk Patients (≥80% risk score):")
    high_risk = summary_df[summary_df['Risk_Score'] >= 80]
    if len(high_risk) > 0:
        for _, patient in high_risk.iterrows():
            print(f"  {patient['Patient_ID']}: {patient['Risk_Score']}% risk")
    else:
        print("  No high-risk patients identified")
    
    return summary_df


In [51]:
custom_path=path=r"C:\Users\ashish.kumar1\CSL Classification\my_ml_projects"

In [52]:
# Load them late
loaded_results = safe_load_results(custom_path)

Loading results from: C:\Users\ashish.kumar1\CSL Classification\my_ml_projects
Experiment: C:\Users\ashish.kumar1\CSL Classification\my_ml_projects
Models: ['RandomForest', 'XGBoost', 'LightGBM', 'LogisticRegression', 'GradientBoosting']
  Loading RandomForest...
    ✅ Models loaded
    ✅ Arrays loaded
    ✅ Metrics loaded
    ✅ Calibration metrics loaded
    ✅ Risk scores loaded
    ✅ Feature importance loaded (5 components)
  Loading XGBoost...
    ✅ Models loaded
    ✅ Arrays loaded
    ✅ Metrics loaded
    ✅ Calibration metrics loaded
    ✅ Risk scores loaded
    ✅ Feature importance loaded (5 components)
  Loading LightGBM...
    ✅ Models loaded
    ✅ Arrays loaded
    ✅ Metrics loaded
    ✅ Calibration metrics loaded
    ✅ Risk scores loaded
    ✅ Feature importance loaded (5 components)
  Loading LogisticRegression...
    ✅ Models loaded
    ✅ Arrays loaded
    ✅ Metrics loaded
    ✅ Calibration metrics loaded
    ✅ Risk scores loaded
    ✅ Feature importance loaded (5 component

In [72]:
def predict_single_patient_risk(patient_data, trained_results, model_name='best_model'):
    """
    Predict risk score for a single patient using calibrated probabilities.
    
    Parameters:
    -----------
    patient_data : pandas.DataFrame or numpy.array
        Single patient's feature data (1 row)
    trained_results : dict
        Results from train_and_evaluate_models_with_calibration()
    model_name : str
        Name of the model to use for prediction
    
    Returns:
    --------
    dict : Comprehensive risk assessment for the patient
    """
    
    # Get the calibrated model
    if model_name == 'best_model':
        # Automatically select best performing model based on calibrated AUC
        best_model = max(trained_results.keys(), 
                        key=lambda x: trained_results[x].get('test_auc_calibrated', 0))
        model_name = best_model
    
    calibrated_model = trained_results[model_name]['calibrated_model']
    
    # Get predictions
    raw_probabilities = trained_results[model_name]['model'].predict_proba(patient_data)
    calibrated_probabilities = calibrated_model.predict_proba(patient_data)
    predicted_class = calibrated_model.predict(patient_data)
    
    # Calculate risk scores
    max_prob = np.max(calibrated_probabilities, axis=1)[0]
    predicted_class_idx = predicted_class[0]
    
    # Risk categorization
    if max_prob >= 0.8:
        risk_category = "High Risk"
        risk_description = "Strong evidence for PID - immediate clinical attention recommended"
    elif max_prob >= 0.6:
        risk_category = "Medium Risk"  
        risk_description = "Moderate evidence for PID - further evaluation recommended"
    elif max_prob >= 0.4:
        risk_category = "Low Risk"
        risk_description = "Limited evidence for PID - consider monitoring"
    else:
        risk_category = "Very Low Risk"
        risk_description = "Minimal evidence for PID - routine follow-up"
    
    # Create comprehensive risk report
    risk_report = {
        'patient_id': f"Patient_{hash(str(patient_data.iloc[0].values)) % 10000}",
        'model_used': model_name,
        
        # Main Risk Assessment
        'risk_score': round(max_prob * 100, 1),  # Convert to percentage
        'risk_category': risk_category,
        'risk_description': risk_description,
        'predicted_class': predicted_class_idx,
        
        # Detailed Probabilities
        'calibrated_probabilities': {
            f'Class_{i}': round(prob * 100, 1) 
            for i, prob in enumerate(calibrated_probabilities[0])
        },

        'raw_probabilities': {
            f'Class_{i}': round(prob * 100, 1) 
            for i, prob in enumerate(raw_probabilities[0])
        },
        
        # Clinical Interpretation
        'confidence_level': get_confidence_interpretation(max_prob),
        'recommendation': get_clinical_recommendation(max_prob, predicted_class_idx),

         ## Top 3 imp features based on impotance df 
        'top Features': trained_results[model_name]['feature_importance']['importance_df'].head(3).to_dict(orient='records'),

        
        
        # Model Performance Context
        'model_performance': {
            # 'model_accuracy': round(trained_results[model_name]['test_accuracy'] * 100, 1),
            # 'model_f1_score': round(trained_results[model_name]['test_f1'] * 100, 1),
            # 'calibration_quality': get_calibration_quality(trained_results[model_name]),
            # 'accuracy_out_of_10': get_accuracy_out_of_10(trained_results[model_name]['test_accuracy']),
            # 'precision_out_of_10': get_accuracy_out_of_10(trained_results[model_name]['test_precision']),
            # 'recall_out_of_10': get_accuracy_out_of_10(trained_results[model_name]['test_recall']),
            'balanced_accuracy': get_accuracy_out_of_10(trained_results[model_name]['test_balanced_accuracy']),
        }
    }
    
    return risk_report


def get_accuracy_out_of_10(metric_value):
    """Convert decimal accuracy/precision/recall to 'out of 10' format."""
    out_of_10 = round(metric_value * 10)
    return f"{out_of_10} out of 10 Patients are predicted correctly"


def get_detailed_performance_interpretation(metric_value, metric_name):
    """Provide detailed interpretation of model performance metrics."""
    out_of_10 = round(metric_value * 10)
    percentage = round(metric_value * 100, 1)
    
    interpretations = {
        'accuracy': f"Correctly predicts {out_of_10} out of 10 cases overall",
        'precision': f"When predicting PID, {out_of_10} out of 10 predictions are correct",
        'recall': f"Identifies {out_of_10} out of 10 actual PID cases",
        'f1': f"Balanced performance score: {percentage}%"
    }
    
    return {
        'out_of_10': f"{out_of_10} out of 10",
        'percentage': f"{percentage}%",
        'interpretation': interpretations.get(metric_name, f"{out_of_10} out of 10")
    }
def get_confidence_interpretation(probability):
    
    """Convert probability to confidence interpretation."""
    if probability >= 0.9:
        return "Very High Confidence"
    elif probability >= 0.8:
        return "High Confidence"
    elif probability >= 0.7:
        return "Moderate Confidence"
    elif probability >= 0.6:
        return "Fair Confidence"
    else:
        return "Low Confidence"


def get_clinical_recommendation(probability, predicted_class):
    """Provide clinical recommendations based on risk score."""
    if probability >= 0.8:
        return "Immediate clinical evaluation recommended. Consider antibiotic treatment."
    elif probability >= 0.6:
        return "Further diagnostic testing recommended (e.g., pelvic exam, lab tests)."
    elif probability >= 0.4:
        return "Monitor symptoms closely. Consider follow-up if symptoms persist."
    else:
        return "Routine follow-up. Educate patient on when to seek care."


def get_calibration_quality(model_results):
    """Assess calibration quality of the model."""
    brier_improvement = model_results['calibration_metrics'].get('brier_improvement', 0)
    if brier_improvement > 0.05:
        return "Excellent calibration improvement"
    elif brier_improvement > 0.02:
        return "Good calibration improvement"
    elif brier_improvement > 0.01:
        return "Fair calibration improvement"
    else:
        return "Minimal calibration improvement"


def print_patient_risk_report(risk_report):
    """Print a formatted risk report for a patient."""
    print("="*80)
    print(f"PID RISK ASSESSMENT REPORT")
    print("="*80)
    print(f"Patient ID: {risk_report['patient_id']}")
    print(f"Model Used: {risk_report['model_used']}")
    print()
    
    print("RISK ASSESSMENT:")
    print(f"  Risk Score: {risk_report['risk_score']}%")
    print(f"  Risk Category: {risk_report['risk_category']}")
    print(f"  Confidence: {risk_report['confidence_level']}")
    print()
    
    print("CLINICAL INTERPRETATION:")
    print(f"  {risk_report['risk_description']}")
    print()
    
    print("RECOMMENDATION:")
    print(f"  {risk_report['recommendation']}")
    print()
    
    print("DETAILED PROBABILITIES:")
    print("  Calibrated Probabilities:")
    for class_name, prob in risk_report['calibrated_probabilities'].items():
        print(f"    {class_name}: {prob}%")
    print()

# Top 3 Features
    print("\n🔍 Top 3 Important Features:")
    print(f"{'Rank':<5} {'Feature':<20} {'Importance':<10}")
    print("-" * 40)
    for idx, feature in enumerate(risk_report['top Features'], start=1):
        print(f"{idx:<5} {feature['feature']:<20} {feature['importance']:<10.4f}")
    
    print("MODEL PERFORMANCE CONTEXT:")
    perf = risk_report['model_performance']
    print(f"balanced_accuracy: {perf['balanced_accuracy']}")
    # print(f"  Model Accuracy: {perf['model_accuracy']}%")
    # print(f"  Model F1-Score: {perf['model_f1_score']}%")
    # print(f"  Calibration Quality: {perf['calibration_quality']}")
    print("="*80)


def batch_predict_patients(patients_data, trained_results, model_name='best_model'):
    """
    Predict risk scores for multiple patients.
    
    Parameters:
    -----------
    patients_data : pandas.DataFrame
        Multiple patients' feature data
    trained_results : dict
        Results from train_and_evaluate_models_with_calibration()
    model_name : str
        Name of the model to use for prediction
    
    Returns:
    --------
    list : List of risk reports for each patient
    """
    risk_reports = []
    
    for idx in range(len(patients_data)):
        patient_data = patients_data.iloc[idx:idx+1]  # Keep as DataFrame
        risk_report = predict_single_patient_risk(patient_data, trained_results, model_name)
        risk_report['patient_index'] = idx
        risk_reports.append(risk_report)
    
    return risk_reports


def create_risk_summary_dashboard(risk_reports):
    """Create a summary dashboard of multiple patient risk assessments."""
    import pandas as pd
    
    # Extract key metrics
    summary_data = []
    for report in risk_reports:
        summary_data.append({
            'Patient_ID': report['patient_id'],
            'Risk_Score': report['risk_score'],
            'Risk_Category': report['risk_category'],
            'Confidence': report['confidence_level'],
            'Predicted_Class': report['predicted_class']
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Risk distribution
    risk_distribution = summary_df['Risk_Category'].value_counts()
    
    print("PATIENT RISK SUMMARY DASHBOARD")
    print("="*60)
    print(f"Total Patients Assessed: {len(risk_reports)}")
    print()
    print("Risk Distribution:")
    for category, count in risk_distribution.items():
        percentage = (count / len(risk_reports)) * 100
        print(f"  {category}: {count} patients ({percentage:.1f}%)")
    print()
    
    print("High Risk Patients (≥80% risk score):")
    high_risk = summary_df[summary_df['Risk_Score'] >= 80]
    if len(high_risk) > 0:
        for _, patient in high_risk.iterrows():
            print(f"  {patient['Patient_ID']}: {patient['Risk_Score']}% risk")
    else:
        print("  No high-risk patients identified")
    
    return summary_df

In [73]:
#get the best model out of the all model based on balace accuracy 
best_model_name = max(loaded_results.keys(),
                       key=lambda x: loaded_results[x]['test_balanced_accuracy'])
best_model_name

'LogisticRegression'

In [76]:
### Feed the patient data to the model
# Load a single patient's data
patient_data=pd.read_csv(r"C:\Users\ashish.kumar1\CSL Classification\single_patient_data.csv")

# Predict risk for the single patient
final_result=predict_single_patient_risk(patient_data, loaded_results, model_name=best_model_name)

c:\Users\ashish.kumar1\CSL Classification\classification\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\ashish.kumar1\CSL Classification\classification\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\ashish.kumar1\CSL Classification\classification\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\ashish.kumar1\CSL Classification\classification\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\ashish.kumar1\CSL Classification\classification\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature 

In [77]:
## get the id,risk score, risk category, confidence level, predicted class, top 3 features,recommendation, model performance 
for keys in final_result.keys():
    if keys in ['patient_id', 'risk_score', 'risk_category', 'confidence_level', 
                          'predicted_class', 'top Features', 'recommendation', 
                          'model_performance']:
        print(f"{keys}: {final_result[keys]}")

patient_id: Patient_837
risk_score: 97.2
risk_category: High Risk
predicted_class: 0
confidence_level: Very High Confidence
recommendation: Immediate clinical evaluation recommended. Consider antibiotic treatment.
top Features: [{'feature': 'PX_STEM_CELL_TRANSPLANT', 'importance': 2.7347331804581523}, {'feature': 'MED_CYTOTOXIC_CHEMOTHERAPY', 'importance': 2.680039046828346}, {'feature': 'CD_HEMATOLOGIC_MALIGNANCIES', 'importance': 2.642911822075066}]
model_performance: {'balanced_accuracy': '8 out of 10 Patients are predicted correctly'}
